In [186]:
import random
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import copy

states = ['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 
          's18', 's19', 's20', 's21', 's22', 's23', 's24']

actions = ['a1', 'a2', 'a3', 'a4'] #up, down, right, left

transition = {
    's0': {
        'a1': {'s0': 0.95, 's1': 0.05},
        'a2': {'s5': 0.8, 's1': 0.05, 's0': 0.15},
        'a3': {'s1': 0.8, 's0': 0.15, 's5': 0.05},
        'a4': {'s0': 0.95, 's5': 0.05}
    },
    's1': {
        'a1': {'s1': 0.9, 's2': 0.05, 's0': 0.05},
        'a2': {'s6': 0.8, 's2': 0.05, 's0': 0.05, 's1': 0.1},
        'a3': {'s2': 0.8, 's1': 0.15, 's6': 0.05},
        'a4': {'s0': 0.8, 's1': 0.15, 's6': 0.05}
    },
    's2': {
        'a1': {'s2': 0.9, 's3': 0.05, 's1': 0.05},
        'a2': {'s7': 0.8, 's3': 0.05, 's1': 0.05, 's2': 0.1},
        'a3': {'s3': 0.8, 's2': 0.15, 's7': 0.05},
        'a4': {'s1': 0.8, 's2': 0.15, 's7': 0.05}
    },
    's3': {
        'a1': {'s3': 0.9, 's4': 0.05, 's2': 0.05},
        'a2': {'s8': 0.8, 's4': 0.05, 's2': 0.05, 's3': 0.1},
        'a3': {'s4': 0.8, 's3': 0.15, 's8': 0.05},
        'a4': {'s2': 0.8, 's3': 0.15, 's8': 0.05}
    },
    's4': {
        'a1': {'s4': 0.95, 's3': 0.05},
        'a2': {'s9': 0.8, 's4': 0.15, 's3': 0.05},
        'a3': {'s4': 0.95, 's9': 0.05},
        'a4': {'s3': 0.8, 's4': 0.15, 's9': 0.05}
    },
    's5': {
        'a1': {'s0': 0.8, 's6': 0.05, 's5': 0.15},
        'a2': {'s10': 0.8, 's6': 0.05, 's5': 0.15},
        'a3': {'s6': 0.8, 's0': 0.05, 's10': 0.05, 's5': 0.1},
        'a4': {'s5': 0.9, 's0': 0.05, 's10': 0.05}
    },
    's6': {
        'a1': {'s1': 0.8, 's7': 0.05, 's5': 0.05, 's6': 0.1},
        'a2': {'s11': 0.8, 's7': 0.05, 's5': 0.05, 's6': 0.1},
        'a3': {'s7': 0.8, 's1': 0.05, 's11': 0.05, 's6': 0.1},
        'a4': {'s5': 0.8, 's1': 0.05, 's11': 0.05, 's6': 0.1}
    },
    's7': {
        'a1': {'s2': 0.8, 's8': 0.05, 's6': 0.05, 's7': 0.1},
        'a2': {'s7': 0.9, 's8': 0.05, 's6': 0.05},
        'a3': {'s8': 0.8, 's2': 0.05, 's7': 0.15},
        'a4': {'s6': 0.8, 's2': 0.05, 's7': 0.15}
    },
    's8': {
        'a1': {'s3': 0.8, 's9': 0.05, 's7': 0.05, 's8': 0.1},
        'a2': {'s13': 0.8, 's9': 0.05, 's7': 0.05, 's8': 0.1},
        'a3': {'s9': 0.8, 's3': 0.05, 's13': 0.05, 's8': 0.1},
        'a4': {'s7': 0.8, 's3': 0.05, 's13': 0.05, 's8': 0.1}
    },
    's9': {
        'a1': {'s4': 0.8, 's9': 0.15, 's8': 0.05},
        'a2': {'s14': 0.8, 's9': 0.15, 's8': 0.05},
        'a3': {'s9': 0.9, 's4': 0.05, 's14': 0.05},
        'a4': {'s8': 0.8, 's4': 0.05, 's14': 0.05, 's9': 0.1}
    },
    's10': {
        'a1': {'s5': 0.8, 's11': 0.05, 's10': 0.15},
        'a2': {'s15': 0.8, 's11': 0.05, 's10': 0.15},
        'a3': {'s11': 0.8, 's5': 0.05, 's15': 0.05, 's10': 0.1},
        'a4': {'s10': 0.9, 's5': 0.05, 's15': 0.05}
    },
    's11': {
        'a1': {'s6': 0.8, 's11': 0.15, 's10': 0.05},
        'a2': {'s16': 0.8, 's11': 0.15, 's10': 0.05},
        'a3': {'s11': 0.9, 's6': 0.05, 's16': 0.05},
        'a4': {'s10': 0.8, 's6': 0.05, 's16': 0.05, 's11': 0.1}
    },
    's12': {
        'a1': {'s7': 0.8, 's13': 0.05, 's11': 0.05, 's12': 0.1},
        'a2': {'s12': 0.9, 's13': 0.05, 's11': 0.05},
        'a3': {'s13': 0.8, 's7': 0.05, 's12': 0.15},
        'a4': {'s11': 0.8, 's7': 0.05, 's12': 0.15}
    },
    's13': {
        'a1': {'s8': 0.8, 's14': 0.05, 's13': 0.15},
        'a2': {'s18': 0.8, 's14': 0.05, 's13': 0.15},
        'a3': {'s14': 0.8, 's8': 0.05, 's18': 0.05, 's13': 0.1},
        'a4': {'s13': 0.9, 's8': 0.05, 's18': 0.05}
    },
    's14': {
        'a1': {'s9': 0.8, 's14': 0.15, 's13': 0.05},
        'a2': {'s19': 0.8, 's14': 0.15, 's13': 0.05},
        'a3': {'s14': 0.9, 's9': 0.05, 's19': 0.05},
        'a4': {'s13': 0.8, 's9': 0.05, 's19': 0.05, 's14': 0.1}
    },
    's15': {
        'a1': {'s10': 0.8, 's16': 0.05, 's15': 0.15},
        'a2': {'s20': 0.8, 's16': 0.05, 's15': 0.15},
        'a3': {'s16': 0.8, 's10': 0.05, 's20': 0.05, 's15': 0.1},
        'a4': {'s15': 0.9, 's10': 0.05, 's20': 0.05}
    },
    's16': {
        'a1': {'s11': 0.8, 's16': 0.15, 's15': 0.05},
        'a2': {'s21': 0.8, 's16': 0.15, 's15': 0.05},
        'a3': {'s16': 0.9, 's11': 0.05, 's21': 0.05},
        'a4': {'s15': 0.8, 's11': 0.05, 's21': 0.05, 's16': 0.1}
    },
    's17': {
        'a1': {'s17': 0.9, 's18': 0.05, 's16': 0.05},
        'a2': {'s22': 0.8, 's18': 0.05, 's16': 0.05, 's17': 0.1},
        'a3': {'s18': 0.8, 's17': 0.15, 's22': 0.05},
        'a4': {'s16': 0.8, 's17': 0.15, 's22': 0.05}
    },
    's18': {
        'a1': {'s13': 0.8, 's19': 0.05, 's18': 0.15},
        'a2': {'s23': 0.8, 's19': 0.05, 's18': 0.15},
        'a3': {'s19': 0.8, 's13': 0.05, 's23': 0.05, 's18': 0.1},
        'a4': {'s18': 0.9, 's13': 0.05, 's23': 0.05}
    },
    's19': {
        'a1': {'s14': 0.8, 's19': 0.15, 's18': 0.05},
        'a2': {'s24': 0.8, 's14': 0.15, 's18': 0.05},
        'a3': {'s19': 0.9, 's14': 0.05, 's24': 0.05},
        'a4': {'s18': 0.8, 's14': 0.05, 's24': 0.05, 's19': 0.1}
    },
    's20': {
        'a1': {'s15': 0.8, 's21': 0.05, 's20': 0.15},
        'a2': {'s20': 0.95, 's21': 0.05},
        'a3': {'s21': 0.8, 's15': 0.05, 's20': 0.15},
        'a4': {'s20': 0.95, 's15': 0.05}
    },
    's21': {
        'a1': {'s16': 0.8, 's22': 0.05, 's20': 0.05, 's21': 0.1},
        'a2': {'s21': 0.9, 's22': 0.05, 's20': 0.05},
        'a3': {'s22': 0.8, 's16': 0.05, 's21': 0.15},
        'a4': {'s20': 0.8, 's16': 0.05, 's21': 0.15}
    },
    's22': {
        'a1': {'s22': 0.9, 's23': 0.05, 's21': 0.05},
        'a2': {'s22': 0.9, 's23': 0.05, 's21': 0.05},
        'a3': {'s23': 0.8, 's22': 0.2},
        'a4': {'s21': 0.8, 's22': 0.2}
    },
    's23': {
        'a1': {'s18': 0.8, 's24': 0.05, 's22': 0.05, 's23': 0.1},
        'a2': {'s23': 0.9, 's24': 0.05, 's22': 0.05},
        'a3': {'s24': 0.8, 's18': 0.05, 's23': 0.15},
        'a4': {'s22': 0.8, 's18': 0.05, 's23': 0.15}
    },
    's24': {
        'a1': {'s19': 0.8, 's24': 0.15, 's23': 0.05},
        'a2': {'s24': 0.95, 's23': 0.05},
        'a3': {'s24': 0.95, 's19': 0.05},
        'a4': {'s23': 0.8, 's19': 0.05, 's24': 0.15}
    }
}
reward_dict = {
    's0': {
        'a1': {'s0': 0, 's1': 0},
        'a2': {'s5': 0, 's1': 0, 's0': 0},
        'a3': {'s1': 0, 's0': 0, 's5': 0},
        'a4': {'s0': 0, 's5': 0}
    },
    's1': {
        'a1': {'s1': 0, 's2': 0, 's0': 0},
        'a2': {'s6': 0, 's2': 0, 's0': 0, 's1': 0},
        'a3': {'s2': 0, 's1': 0, 's6': 0},
        'a4': {'s0': 0, 's1': 0, 's6': 0}
    },
    's2': {
        'a1': {'s2': 0, 's3': 0, 's1': 0},
        'a2': {'s7': 0, 's3': 0, 's1': 0, 's2': 0},
        'a3': {'s3': 0, 's2': 0, 's7': 0},
        'a4': {'s1': 0, 's2': 0, 's7': 0}
    },
    's3': {
        'a1': {'s3': 0, 's4': 0, 's2': 0},
        'a2': {'s8': 0, 's4': 0, 's2': 0, 's3': 0},
        'a3': {'s4': 0, 's3': 0, 's8': 0},
        'a4': {'s2': 0, 's3': 0, 's8': 0}
    },
    's4': {
        'a1': {'s4': 0, 's3': 0},
        'a2': {'s9': 0, 's4': 0, 's3': 0},
        'a3': {'s4': 0, 's9': 0},
        'a4': {'s3': 0, 's4': 0, 's9': 0}
    },
    's5': {
        'a1': {'s0': 0, 's6': 0, 's5': 0},
        'a2': {'s10': 0, 's6': 0, 's5': 0},
        'a3': {'s6': 0, 's0': 0, 's10': 0, 's5': 0},
        'a4': {'s5': 0, 's0': 0, 's10': 0}
    },
    's6': {
        'a1': {'s1': 0, 's7': 0, 's5': 0, 's6': 0},
        'a2': {'s11': 0, 's7': 0, 's5': 0, 's6': 0},
        'a3': {'s7': 0, 's1': 0, 's11': 0, 's6': 0},
        'a4': {'s5': 0, 's1': 0, 's11': 0, 's6': 0}
    },
    's7': {
        'a1': {'s2': 0, 's8': 0, 's6': 0, 's7': 0},
        'a2': {'s7': 0, 's8': 0, 's6': 0},
        'a3': {'s8': 0, 's2': 0, 's7': 0},
        'a4': {'s6': 0, 's2': 0, 's7': 0}
    },
    's8': {
        'a1': {'s3': 0, 's9': 0, 's7': 0, 's8': 0},
        'a2': {'s13': 0, 's9': 0, 's7': 0, 's8': 0},
        'a3': {'s9': 0, 's3': 0, 's13': 0, 's8': 0},
        'a4': {'s7': 0, 's3': 0, 's13': 0, 's8': 0}
    },
    's9': {
        'a1': {'s4': 0, 's9': 0, 's8': 0},
        'a2': {'s14': 0, 's9': 0, 's8': 0},
        'a3': {'s9': 0, 's4': 0, 's14': 0},
        'a4': {'s8': 0, 's4': 0, 's14': 0, 's9': 0}
    },
    's10': {
        'a1': {'s5': 0, 's11': 0, 's10': 0},
        'a2': {'s15': 0, 's11': 0, 's10': 0},
        'a3': {'s11': 0, 's5': 0, 's15': 0, 's10': 0},
        'a4': {'s10': 0, 's5': 0, 's15': 0}
    },
    's11': {
        'a1': {'s6': 0, 's11': 0, 's10': 0},
        'a2': {'s16': 0, 's11': 0, 's10': 0},
        'a3': {'s11': 0, 's6': 0, 's16': 0},
        'a4': {'s10': 0, 's6': 0, 's16': 0, 's11': 0}
    },
    's12': {
        'a1': {'s7': 0, 's13': 0, 's11': 0, 's12': 0},
        'a2': {'s12': 0, 's13': 0, 's11': 0},
        'a3': {'s13': 0, 's7': 0, 's12': 0},
        'a4': {'s11': 0, 's7': 0, 's12': 0}
    },
    's13': {
        'a1': {'s8': 0, 's14': 0, 's13': 0},
        'a2': {'s18': 0, 's14': 0, 's13': 0},
        'a3': {'s14': 0, 's8': 0, 's18': 0, 's13': 0},
        'a4': {'s13': 0, 's8': 0, 's18': 0}
    },
    's14': {
        'a1': {'s9': 0, 's14': 0, 's13': 0},
        'a2': {'s19': 0, 's14': 0, 's13': 0},
        'a3': {'s14': 0, 's9': 0, 's19': 0},
        'a4': {'s13': 0, 's9': 0, 's19': 0, 's14': 0}
    },
    's15': {
        'a1': {'s10': 0, 's16': 0, 's15': 0},
        'a2': {'s20': 0, 's16': 0, 's15': 0},
        'a3': {'s16': 0, 's10': 0, 's20': 0, 's15': 0},
        'a4': {'s15': 0, 's10': 0, 's20': 0}
    },
    's16': {
        'a1': {'s11': 0, 's16': 0, 's15': 0},
        'a2': {'s21': 0, 's16': 0, 's15': 0},
        'a3': {'s16': 0, 's11': 0, 's21': 0},
        'a4': {'s15': 0, 's11': 0, 's21': 0, 's16': 0}
    },
    's17': {
        'a1': {'s17': 0, 's18': 0, 's16': 0},
        'a2': {'s22': -10, 's18': 0, 's16': 0, 's17': 0},
        'a3': {'s18': 0, 's17': 0, 's22': -10},
        'a4': {'s16': 0, 's17': 0, 's22': -10}
    },
    's18': {
        'a1': {'s13': 0, 's19': 0, 's18': 0},
        'a2': {'s23': 0, 's19': 0, 's18': 0},
        'a3': {'s19': 0, 's13': 0, 's23': 0, 's18': 0},
        'a4': {'s18': 0, 's13': 0, 's23': 0}
    },
    's19': {
        'a1': {'s14': 0, 's19': 0, 's18': 0},
        'a2': {'s24': 10, 's14': 0, 's18': 0},
        'a3': {'s19': 0, 's14': 0, 's24': 10},
        'a4': {'s18': 0, 's14': 0, 's24': 10, 's19': 0}
    },
    's20': {
        'a1': {'s15': 0, 's21': 0, 's20': 0},
        'a2': {'s20': 0, 's21': 0},
        'a3': {'s21': 0, 's15': 0, 's20': 0},
        'a4': {'s20': 0, 's15': 0}
    },
    's21': {
        'a1': {'s16': 0, 's22': -10, 's20': 0, 's21': 0},
        'a2': {'s21': 0, 's22': -10, 's20': 0},
        'a3': {'s22': -10, 's16': 0, 's21': 0},
        'a4': {'s20': 0, 's16': 0, 's21': 0}
    },
    's22': {
        'a1': {'s22': -10, 's23': 0, 's21': 0},
        'a2': {'s22': -10, 's23': 0, 's21': 0},
        'a3': {'s23': 0, 's22': -10},
        'a4': {'s21': 0, 's22': -10}
    },
    's23': {
        'a1': {'s18': 0, 's24': 10, 's22': -10, 's23': 0},
        'a2': {'s23': 0, 's24': 10, 's22': -10},
        'a3': {'s24': 10, 's18': 0, 's23': 0},
        'a4': {'s22': -10, 's18': 0, 's23': 0}
    },
    's24': {
        'a1': {'s19': 0, 's24': 0, 's23': 0},
        'a2': {'s24': 0, 's23': 0},
        'a3': {'s24': 0, 's19': 0},
        'a4': {'s23': 0, 's19': 0, 's24': 0}
    }
}

policy_optimal = ['a3', 'a3', 'a3', 'a2', 'a2', 
                  'a3', 'a3', 'a3', 'a2', 'a2', 
                  'a1', 'a1', ' ', 'a2', 'a2', 
                  'a1', 'a1', ' ', 'a2', 'a2', 
                  'a1', 'a1', 'a3', 'a3', 0]

v_optimal = [4.0187, 4.5548, 5.1575, 5.8336, 6.4553, 
             4.3716, 5.0324, 5.8013, 6.6473, 7.3907, 
             3.8672, 4.39, 0.0000, 7.5769, 8.4637, 
             3.4182, 3.8319, 0.0000, 8.5738, 9.6946, 
             2.9977, 2.9309, 6.0733, 9.6946, 0.0000]


def choosing_nextstate(state, action_taken):
    
    possible_states = list(transition[state][action_taken].keys())
    possible_states_probability = list(transition[state][action_taken].values())

    chosen_state = random.choices(possible_states, possible_states_probability)[0]

    return chosen_state

def td_learning():
    v_calculated = np.zeros_like(v_optimal)
    count_episode = 0
    
    
    while True:
        valid_states = [state for state in states if state not in ['s12', 's17']]
        start_state = random.choice(valid_states) #initializing start state
        state = start_state
        
        new_v_calculated = copy.deepcopy(v_calculated)
        
        while state != 's24': #check the loop
            
            action_taken = policy_optimal[int(state[1:])]
            
            next_state = choosing_nextstate(state, action_taken)
            value_nextstate = v_calculated[int(next_state[1:])]
            reward = reward_dict[state][action_taken][next_state]

            new_v_calculated[int(state[1:])] = new_v_calculated[int(state[1:])] + alpha * ((reward + gamma * value_nextstate)-new_v_calculated[int(state[1:])])

            state = next_state
            
        count_episode += 1
        
        max_norm_dist = np.max(np.abs(new_v_calculated - v_calculated))
        v_calculated = copy.deepcopy(new_v_calculated)
    
        if max_norm_dist < theta:
            break
            
    return v_calculated, count_episode

alpha = 0.8
theta = 0.0001
gamma = 0.9
# v_calculated = np.zeros_like(v_optimal)
value_sum = np.zeros_like(v_optimal)
episode_sum = 0
episode_sum_list = []

count_episode = 0
for i in range(50): #set it to 50
    
    each_episode_value_calculated, episode_count = td_learning()

    value_sum = value_sum + each_episode_value_calculated
    episode_sum = episode_sum + episode_count
    episode_sum_list.append(episode_count)

average_value_state = value_sum / 50 #hardcode
max_norm_distance = np.max(np.abs(average_value_state - v_optimal))


average_episode_count = episode_sum / 50
stddev = np.std(episode_sum_list) 

print(f"Average episodes : {round(average_episode_count,4)}")
print(f"Max Norm Distance : {round(max_norm_distance,4)}")
print(f"Standard Deviation : {round(stddev,4)}")

for i in range(0, len(average_value_state), 5):
    row = average_value_state[i:i+5]
    for index, element in enumerate(row):
        if index + i == 12 or index + i == 17:
            print("0.0000 ", end='\t')
        else:
            print(f"{round(element,4)}", end='\t')
    print()

Average episodes : 16.16
Max Norm Distance : 5.285
Standard Deviation : 11.0061
0.295	0.7901	1.7936	3.004	1.1703	
1.262	2.1913	3.4349	4.8035	2.9944	
0.5364	1.1797	0.0000 	6.2039	5.564	
0.2497	0.4208	0.0000 	7.7366	8.549	
0.0612	0.1274	2.6664	9.2123	0.0	
